## Integrantes

- **Estudiante 1:** Juan David Duarte Yara - 202215070
- **Estudiante 2:** Luisa Gabriela Hernández Pérez - 202114093
- **Estudiante 3:** Juan Sebastian Sanchez - 202121498
- Todos los estudiantes aportaron en las partes generales del notebook y en las diapositivas de coclusiones.



## 0. Carga de los datos. 

### 0.1. Importación de librerías 

En las siguientes líneas de código se importan las librerías y herramientas necesarias para desarrollar el caso de uso.

In [44]:
# Importaciones de bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import sys
import re
import string
import unicodedata

# Importaciones de joblib
from joblib import dump, load

# Importaciones de sklearn
from sklearn.base import BaseEstimator, RegressorMixin, ClassifierMixin
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.impute import SimpleImputer
from sklearn import tree
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

# Clasificacion
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification

# Importaciones de statsmodels
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan, het_white

# Importaciones de nltk
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer, SnowballStemmer

# Instalación de paquetes adicionales
import contractions

# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV

# Para la validación cruzada
from sklearn.model_selection import KFold

### 0.2. Lectura de archivos
A través de la librería **pandas** podemos realizar la carga de datos desde diferentes fuentes de información, en este caso se realizará la carga de un archivo plano csv (archivo separado por comas).

In [45]:
# Se cargan los datos. 
datatrain=pd.read_csv('fake_news_spanish.csv', sep=';', encoding = 'utf-8')
datatest = pd.read_csv('fake_news_test.csv', sep=';', encoding = 'utf-8')

Podemos ver que hay 57,063 filas y 5 columnas de atributos en el conjunto de entrenamiento

In [46]:
datatrain.shape

(57063, 5)

Podemos ver que hay 1000 filas y 4 columnas de atributos en el conjunto de prueba

In [47]:
datatest.shape

(1000, 4)

Veamos un pequeño sample de los datos

In [48]:
datatrain.sample(10)

,ID,Label,Titulo,Descripcion,Fecha
55494,ID,1,Arrimadas rechaza la autocrítica tras el porta...,La portavoz nacional del partido anuncia que M...,24/06/2019
19103,ID,1,"Cifuentes, la aspirante a regeneradora que cay...",No ha aguantado ni tres años al frente de la C...,25/04/2018
18608,ID,0,Hermann Tertsch se hace la víctima: el defenso...,El columnista se erige en mártir de la indepen...,04/12/2018
9476,ID,0,El Gobierno encauza la estrategia de apoyo a l...,El Ministerio de Trabajo está cerca de consens...,25/01/2021
53409,ID,1,La Asociación de Periodistas de Investigación ...,La API insta al Ejecutivo de Pedro Sánchez a q...,27/09/2022
18573,ID,1,Unidas Podemos muestra sus grandes discrepanci...,Critica en el seno del Ejecutivo la falta de l...,25/08/2020
33266,ID,1,"Después de Meirás toca la Casa Cornide, el pal...",La Xunta inicia el procedimiento para declarar...,23/05/2021
9839,ID,0,El juez archiva la causa contra Gloria Calero ...,El magistrado García-Castellón ha decretado el...,20/04/2021
50889,ID,0,"Muere Àngel Ros, hermano del expresidente Salv...",Ha fallecido en el Hospital Virgen Macarena a ...,26/05/2019
49304,ID,0,Pablo Echenique recupera las ayudas para los '...,El programa electoral del BNG madrileño no nom...,15/04/2021


Revisemos que el tipo de dato sea el esperado

In [49]:
datatrain.dtypes

ID             object
Label           int64
Titulo         object
Descripcion    object
Fecha          object
dtype: object

# 2. Entendimiento de los datos

## 2.1 Analisis descriptivo

Podemos ver que hay textos repetidos

In [50]:
datatrain["Descripcion"].describe()


count                                                 57063
unique                                                49638
top       La Fundación En Acción ha participado con Públ...
freq                                                      7
Name: Descripcion, dtype: object

Es útil saber que tan largos son los textos

In [51]:
datatrain["Descripcion"].apply(len).describe()

count    57063.000000
mean       243.519724
std        274.479568
min         33.000000
25%        171.000000
50%        221.000000
75%        267.000000
max      14153.000000
Name: Descripcion, dtype: float64

Los textos tienen una longitud entre 33 y 14,153 caracteres, con una media de 243 caracteres. 

Queremos ver ahora la columna "label"

In [52]:
datatrain["Label"].describe()

count    57063.000000
mean         0.581077
std          0.493387
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: Label, dtype: float64

Podemos ver que efectivamente la columna "Label" tiene valores entre 0 y 1.

Nos gustaría saber la proporción que hay entre noticias falsas y las que no

In [53]:
datatrain["Label"].value_counts()

Label
1    33158
0    23905
Name: count, dtype: int64

Podemos ver que los datos están ligeramente desbalanceados

## 2.2 Calidad de datos

In [54]:
# Es recomendable que todos los pasos de limpieza y preparación se realicen sobre otro archivo.
data_t = datatrain.copy()

### Unicidad

Revisemos los duplicados por título 

In [55]:
data_t.loc[datatrain.duplicated(subset= "Titulo", keep = False)].sort_values("Titulo").head(10)

,ID,Label,Titulo,Descripcion,Fecha
53500,ID,1,"""La quimioterapia mata a la gente y no al cánc...",Señor Presidente recibiremos sus tropas con lo...,01/11/2018
27107,ID,1,"""La quimioterapia mata a la gente y no al cánc...",Estamos en la semana decisiva. Los expertos a...,10/08/2021
44905,ID,1,#DespuésDeAplaudir queda mucho por hacer y rei...,Acto a favor de la Sanidad pública y universal...,28/05/2020
41264,ID,0,#DespuésDeAplaudir queda mucho por hacer y rei...,Acto a favor de la Sanidad pública y universal...,28/05/2020
9017,ID,1,'11 millones de euros por teléfono y por Whats...,Los comisionistas del escándalo de las masacri...,26/04/2022
25449,ID,0,'11 millones de euros por teléfono y por Whats...,Los comisionistas del escándalo de las masacri...,26/04/2022
42708,ID,1,'A este hay que matarlo'... Las perlas de las ...,El comisario relata el robo de información al ...,30/09/2020
41717,ID,0,'A este hay que matarlo'... Las perlas de las ...,El comisario relata el robo de información al ...,30/09/2020
26334,ID,1,'A mi madre la llevaron al sótano de la reside...,Comienzan a declarar en el juzgado los familia...,22/02/2021
7823,ID,0,'A mi madre la llevaron al sótano de la reside...,Comienzan a declarar en el juzgado los familia...,22/02/2021


Revisemos ahora los duplicados por descripción 

In [56]:
data_t.loc[datatrain.duplicated(subset= "Descripcion", keep = False)].sort_values("Descripcion").head(10)

,ID,Label,Titulo,Descripcion,Fecha
10367,ID,0,Inma Nieto: 'José Manuel Franco me decía hace ...,(Risas) Sí...lo que pasa es que esto viene ya ...,03/01/2023
51799,ID,1,Edmundo Bal: 'Adrián Vázquez me decía hace poc...,(Risas) Sí...lo que pasa es que esto viene ya ...,03/01/2023
38903,ID,1,Un manifiesto llama a que Xavier Domènech lide...,134 miembros destacados de la formación catala...,02/03/2018
31583,ID,0,Un manifiesto llama a que Adriana Lastra lider...,134 miembros destacados de la formación catala...,02/03/2018
27018,ID,1,¿Rey franquista o padre de la democracia?: luc...,43 años y medio después de jurar los principio...,01/06/2019
14590,ID,0,¿Rey franquista o padre de la democracia?: luc...,43 años y medio después de jurar los principio...,01/06/2019
20779,ID,1,Los médicos de Atención Primaria en Madrid se ...,5.000 sanitarios de los Centros de Salud de la...,10/03/2021
49171,ID,0,Los médicos de Atención Primaria en Madrid se ...,5.000 sanitarios de los Centros de Salud de la...,10/03/2021
27698,ID,1,Procesados Sito Miñanco y Gonzalo Boye por bla...,"A Boye, el abogado de Puigdemont, se le relaci...",18/12/2020
40787,ID,0,Procesados Sito Miñanco y Feijoo por blanqueo ...,"A Boye, el abogado de Puigdemont, se le relaci...",18/12/2020


In [57]:
data_t.duplicated(keep=False).sum()

875

Revisando el archivo pudimos notar que realmente los anteriores datos no son duplicados, sino que tienen ligeras diferencias entre su contenido, reafirmando que uno es una fake news y la otra no.

Ahora revisemos cuando la columna "Titulo", "Descripcción","Fecha" y "Label" son exactamente iguales.

In [58]:
data_t.loc[datatrain.duplicated(subset= ["Descripcion","Titulo"], keep = False)].sort_values("Descripcion").head(10)

,ID,Label,Titulo,Descripcion,Fecha
54191,ID,1,Madrid envía una carta a los directores de los...,"A cinco días del 4M, el director general de Ed...",30/04/2021
54892,ID,1,Madrid envía una carta a los directores de los...,"A cinco días del 4M, el director general de Ed...",30/04/2021
1551,ID,1,"El legado del PP en Madrid: desigualdad, anemi...",A lo largo de la campaña se hablará más en cla...,19/04/2021
8627,ID,1,"El legado del PP en Madrid: desigualdad, anemi...",A lo largo de la campaña se hablará más en cla...,19/04/2021
16683,ID,1,Ayuso recurre a las descalificaciones para def...,A lo largo del debate la dirigente conservador...,21/04/2021
18999,ID,1,Ayuso recurre a las descalificaciones para def...,A lo largo del debate la dirigente conservador...,21/04/2021
55074,ID,1,ERC y JxCat se reúnen en la cárcel de Lledoner...,A menos de un mes para agotar el plazo para in...,27/04/2021
27758,ID,1,ERC y JxCat se reúnen en la cárcel de Lledoner...,A menos de un mes para agotar el plazo para in...,27/04/2021
36567,ID,1,Otra amenaza de muerte sin una contundente con...,Abascal dice que la carta con amenazas a Igles...,26/04/2021
29871,ID,1,Otra amenaza de muerte sin una contundente con...,Abascal dice que la carta con amenazas a Igles...,26/04/2021


Todas estas instancias son exactamente iguales, por lo cual, son duplicados. Revisemos cuantos duplicados realmente hay.

In [59]:
data_t.duplicated(subset=['Titulo', 'Descripcion','Fecha','Label'], keep=False).sum()

875

Borremos los duplicados

In [60]:
data_t = data_t.drop_duplicates(subset=['Titulo', 'Descripcion','Fecha','Label'], keep='first')

Revisemos si el tamaño de los datos cambió. El cambio debe ser consistente con la cantidad de duplicados eliminados

In [61]:
datatrain.shape

(57063, 5)

In [62]:
data_t.shape

(56618, 5)

Revisemos si hay duplicados

In [63]:
data_t.duplicated(subset=['Titulo', 'Descripcion','Fecha','Label'], keep=False).sum()

0

### Completitud

In [64]:
# Se observa que hay ausencias en la columna titulo
data_t.isnull().sum()

ID              0
Label           0
Titulo         16
Descripcion     0
Fecha           0
dtype: int64

In [65]:
# Como solo son 16 instancias de más de 57,000 podemos borrar estos datos
data_t = data_t.dropna(subset=["Titulo"])

In [66]:
# Se observa que ya no hay ausencias en la columna titulo
data_t.isnull().sum()

ID             0
Label          0
Titulo         0
Descripcion    0
Fecha          0
dtype: int64

### Validez

Podemos confirmar que solo existen los labels de fake news y no fake news.

In [67]:
pd.DataFrame(data_t["Label"].unique())

,0
0,1
1,0


### Consistencia

Revisemos el formato de las fechas

In [68]:
data_t["Fecha"].isna().sum()


0

In [69]:
data_t['Nueva_Fecha'] = pd.to_datetime(data_t['Fecha'], format="%d/%m/%Y", errors ='coerce')
data_t['Nueva_Fecha'].sample(10)

42129   2023-04-08
55272   2019-02-15
54132   2022-02-08
16245   2019-10-06
10121   2019-05-06
25622   2020-10-18
36761   2021-09-11
38059   2021-09-21
29583   2020-07-13
8038    2022-08-13
Name: Nueva_Fecha, dtype: datetime64[ns]

In [70]:
# Miramos los registros que no pudieron convertirse a DateTime. Vemos que todos se convirtieron satisfactoriamente a datetime y no hay ningun NaT
data_t.loc[data_t.Nueva_Fecha.isna(), ['Fecha','Nueva_Fecha']] 

,Fecha,Nueva_Fecha


In [71]:
#Revisemos los tipos de datos
data_t.dtypes

ID                     object
Label                   int64
Titulo                 object
Descripcion            object
Fecha                  object
Nueva_Fecha    datetime64[ns]
dtype: object

# 3 Preparación de datos

## 3.1 Procesamiento de texto
Ahora tenemos que procesar el texto para que pueda utilizarlo el algoritmo de clasificación

In [72]:
# Descargar el conjunto de palabras de parada (stopwords) de nltk
nltk.download('stopwords')

# Descargar el tokenizer preentrenado 'punkt' para dividir texto en oraciones y palabras
nltk.download('punkt')

# Descargar el léxico WordNet para la lematización y otras tareas léxicas
nltk.download('wordnet')

# Descargar el Open Multilingual Wordnet (OMW) para soporte multilingüe en WordNet
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\57300\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\57300\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\57300\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\57300\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Las siguientes funciones nos servirán para hacer la limpieza, preprocesamiento y procesamiento de las palabras

In [73]:
def aMinusculas(palabras):
    return [palabra.lower() for palabra in palabras]

def eliminarNumeros(palabras):
    return [re.sub(r'\d+', '', palabra) for palabra in palabras]

def eliminarPuntuacion(palabras):
    return [palabra for palabra in palabras if palabra not in string.punctuation]

def removerNoAscii(palabras):
    return [palabra for palabra in palabras if palabra.isascii()]

def eliminarStopwords(palabras):
    sw = set(stopwords.words('spanish'))
    return [palabra for palabra in palabras if palabra not in sw]

def aplicarStemmingYLematizacion(palabras):
    # Evidenciamos que brinda mejores resultados que PorterStemmer para español
    stemmer = SnowballStemmer('spanish')
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(stemmer.stem(palabra)) for palabra in palabras]

def preprocesar(palabras):
    palabras = aMinusculas(palabras)
    palabras = eliminarNumeros(palabras)
    palabras = eliminarPuntuacion(palabras)
    palabras = removerNoAscii(palabras)
    palabras = eliminarStopwords(palabras)
    return palabras

def procesar(texto):
    texto = contractions.fix(texto)
    palabras = word_tokenize(texto)
    palabras = preprocesar(palabras)
    palabras = aplicarStemmingYLematizacion(palabras)
    return ' '.join(palabras)

A continuación podemos ver el resultado del procesamiento de los textos en la columna "Titulo" y "Descripccion"

In [74]:
data_t["Titulo"] = data_t['Titulo'].apply(procesar)
data_t["Descripcion"] = data_t["Descripcion"].apply(procesar)
data_t

,ID,Label,Titulo,Descripcion,Fecha,Nueva_Fecha
0,ID,1,'the guardi va 'europ necesit apuest frut,diari pa juev editorial eleccion j calif valie...,02/06/2023,2023-06-02
1,ID,0,revel gobiern negoci mirel cambi otorg javi duart,revel gobiern negoci mirel cambi otorg javi du...,01/10/2023,2023-10-01
2,ID,1,'ahor nunc joan fust estatut valencian cumpl,valencian convoc fiest grand aplec plaz tor ca...,25/04/2022,2022-04-25
3,ID,1,iglesi alient yoland erc eh bildu negoci 'rear...,igual negoci empresari negoci grup parlamentar...,03/01/2022,2022-01-03
4,ID,0,puigdemont 'no ningun tragedi eleccion,entrev punt avui jxcat desdramatiz posibl esce...,09/03/2018,2018-03-09
...,...,...,...,...,...,...
57058,ID,1,defensor puebl reclam comun madr dat demor med...,gobiern regional indic inclu inmediat contempl...,08/06/2021,2021-06-08
57059,ID,0,equ plante ced president comun madr c tal echa...,si higien llev exigent ten pa person social pr...,08/09/2020,2020-09-08
57060,ID,1,albert 'qu borbon ladron hech constat,coordin federal iu asegur putrefact,12/07/2018,2018-07-12
57061,ID,1,vox exig entrar gobiern castill car vicepresid...,santiag abascal vox derech deb form gobiern ca...,13/02/2022,2022-02-13


In [75]:
data_t.columns

Index(['ID', 'Label', 'Titulo', 'Descripcion', 'Fecha', 'Nueva_Fecha'], dtype='object')

## 3.2 NM-gramas
Implementamos NM-gramas con un rango de 1 a 3. 

In [ ]:
from scipy.sparse import hstack
# Inicializar el vectorizador con reducción de características
bv = CountVectorizer(ngram_range=(1, 3), min_df=2, max_df=0.9, max_features=5000)

# Procesar "Titulo" (sin convertir a denso)
bv_matrix_titulo = bv.fit_transform(data_t["Titulo"])
vocab_titulo = [f"titulo_{word}" for word in bv.get_feature_names_out()]

# Procesar "Descripcion" (sin convertir a denso)
bv_matrix_desc = bv.fit_transform(data_t["Descripcion"])
vocab_desc = [f"desc_{word}" for word in bv.get_feature_names_out()]

# Combinar ambas matrices dispersas
bv_matrix_total = hstack([bv_matrix_titulo, bv_matrix_desc])

# Convertir la matriz dispersa en DataFrame (evitando el uso de `toarray()`)
bv_df = pd.DataFrame.sparse.from_spmatrix(bv_matrix_total, columns=vocab_titulo + vocab_desc)

# Concatenar con el DataFrame original (sin convertir a matriz densa)
opiniones_train_df = pd.concat([data_t.drop(columns=["Titulo", "Descripcion"]), bv_df], axis=1)



,ID,Label,Fecha,Nueva_Fecha,titulo_abandon,titulo_abascal,titulo_abascal pabl,titulo_abascal pabl echeniqu,titulo_abc,titulo_abengo,...,desc_zamor,desc_zanj,desc_zapater,desc_zaplan,desc_zaragoz,desc_zarzalej,desc_zarzuel,desc_zerol,desc_zoid,desc_zon
0,ID,1.0,02/06/2023,2023-06-02,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ID,0.0,01/10/2023,2023-10-01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ID,1.0,25/04/2022,2022-04-25,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ID,1.0,03/01/2022,2022-01-03,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ID,0.0,09/03/2018,2018-03-09,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
# Mostrar las primeras filas del nuevo DataFrame
opiniones_train_df.sample(10)

,ID,Label,Fecha,Nueva_Fecha,titulo_abandon,titulo_abascal,titulo_abascal pabl,titulo_abascal pabl echeniqu,titulo_abc,titulo_abengo,...,desc_zamor,desc_zanj,desc_zapater,desc_zaplan,desc_zaragoz,desc_zarzalej,desc_zarzuel,desc_zerol,desc_zoid,desc_zon
32986,ID,1.0,10/10/2019,2019-10-10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7632,ID,1.0,06/10/2019,2019-10-06,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52569,ID,1.0,13/02/2018,2018-02-13,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21312,ID,0.0,11/03/2019,2019-03-11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24853,ID,1.0,04/02/2019,2019-02-04,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44170,ID,0.0,12/04/2019,2019-04-12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
334,ID,1.0,25/01/2018,2018-01-25,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8921,ID,1.0,17/03/2020,2020-03-17,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7889,ID,1.0,13/07/2020,2020-07-13,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42377,ID,1.0,06/01/2023,2023-01-06,0,0,0,0,0,0,...,0,0,0,0,1.0,0,0,0,0,0


### 3.4. Creación y mejora de los conjuntos de entrenamiento y validación

In [86]:
# Se selecciona la variable objetivo
Y = opiniones_train_df[['Label']]
# Del conjunto de datos se eliminan las variables objetivo
X = opiniones_train_df.drop(['Label', "ID","Fecha", "Nueva_Fecha"], axis=1)

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.2, random_state=0)
X_train

,titulo_abandon,titulo_abascal,titulo_abascal pabl,titulo_abascal pabl echeniqu,titulo_abc,titulo_abengo,titulo_abertzal,titulo_abiert,titulo_aboc,titulo_abog,...,desc_zamor,desc_zanj,desc_zapater,desc_zaplan,desc_zaragoz,desc_zarzalej,desc_zarzuel,desc_zerol,desc_zoid,desc_zon
39812,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55449,0,0,0,0,0,0,0,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
33143,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20656,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52826,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42903,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:

print("NaN en X_train:", np.isnan(X_train).sum().sum())
print("Inf en X_train:", np.isinf(X_train).sum().sum())


NaN en X_train: 10000
Inf en X_train: 0


In [93]:
X_train = X_train.dropna()


In [94]:
print("NaN en X_train:", np.isnan(X_train).sum().sum())
print("Inf en X_train:", np.isinf(X_train).sum().sum())

NaN en X_train: 0
Inf en X_train: 0


In [99]:
print("NaN en Y_train:", Y_train.isna().sum())

Y_train = Y_train.dropna()

NaN en Y_train: Label    0
dtype: int64


In [101]:
X_train = X_train.loc[Y_train.index]
Y_train = Y_train.loc[X_train.index]


In [102]:
smt = SMOTE()
X_smote, Y_smote = smt.fit_resample(X_train, Y_train)
X_smote

,titulo_abandon,titulo_abascal,titulo_abascal pabl,titulo_abascal pabl echeniqu,titulo_abc,titulo_abengo,titulo_abertzal,titulo_abiert,titulo_aboc,titulo_abog,...,desc_zamor,desc_zanj,desc_zapater,desc_zaplan,desc_zaragoz,desc_zarzalej,desc_zarzuel,desc_zerol,desc_zoid,desc_zon
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
scaler = StandardScaler()
X_smote = pd.DataFrame(scaler.fit_transform(X_smote), columns=vocab_titulo)
X_validation = pd.DataFrame(scaler.fit_transform(X_validation), columns=vocab_titulo)
X_smote

MemoryError: Unable to allocate 3.90 GiB for an array with shape (10000, 52352) and data type float64